In [1]:
# RUN WITH APPMODE!!

In [2]:
import os
import time
import dateutil
import datetime
import shutil
import glob
import rdflib
import pandas as pd
from ipywidgets import Button, Label, HBox, Output
from utilities import hydroshare
import qgrid
from collections import OrderedDict
os.chdir(os.path.expanduser('~/notebooks'))

In [3]:
%%html
<style>
.mytext {
    font-style: bold;
    font-size: 20px;
    padding: 25px 25px 25px 25px;
}
</style>

In [4]:
sel_dir = ''
op = None

# table selection changed
def select_cb(event, w):
    global sel_dir
    
    if event['new'] == []:
        for b in buttons:
            b.disabled = True
        new_button.disabled = False   
        return

    outbox.clear_output()
    
    if sel_dir:
        # cancel previous operation
        hbox.children = buttons
        sel_dir = ''

    # enable all buttons
    for b in buttons:
        b.disabled = False

    # disable UPDATE button if no update available
    status = qw.get_selected_df()['Status'][0]
    if status.split('"')[3] != 'Modified':
        update_button.disabled = True
    # disable SEND button if no hydroshare resource exists
    if status.split('"')[3] == 'Deleted':
        send_button.disabled = True

del_button = Button (
                description='Delete Local Copy',
                icon='minus',
                tooltip='Delete Local Copy of Resource and its Files',
                button_style='danger',
                disabled=True
            )
send_button = Button (
                description='Send to HydroShare',
                icon='fa-arrow-left',
                tooltip='Send your file to the resource on HydroShare',
                button_style='warning',
                layout={'width': 'initial'},
                disabled=True
            )
update_button = Button (
                description='Update from HydroShare',
                icon='fa-arrow-right',
                tooltip='Fetch the most recent version of this resource form HydroShare',
                button_style='success',
                layout={'width': 'initial'},
                disabled=True
            )
copy_button = Button (
                description='New Copy',
                icon='plus',
                tooltip='Create a new resource on HydroShare from this resource',
                button_style='info',
                disabled=True
            )
new_button = Button (
                description='New Resource',
                icon='plus-square',
                tooltip='Create a new (empty) resource on HydroShare.',
                #button_style='info',
                disabled=False
            )
confirm_label = Label(value='')
cancel_but = Button(description='Cancel', button_style='info')
ok_but = Button(description='OK', button_style='danger')
confirm_dialog = [confirm_label, cancel_but, ok_but]
buttons = [del_button, update_button, send_button, copy_button, new_button]

In [5]:
def get_del_dir(d):
    # d is path to contents
    d = os.path.dirname(os.path.dirname(d))
    rid = os.path.basename(d)
    rid2 = os.path.basename(os.path.dirname(d))
    if rid == rid2:
        d = os.path.dirname(d)
    return d

def but_clicked(w):
    # delete clicked.  bring up confirm dialog
    global sel_dir, op
    #del_button.disabled = True
    ok_but.disabled = False
    cancel_but.disabled = False
    
    if w == new_button:
        confirm_label.value = 'Create a new (empty) resource?'
        op = new_project
        hbox.children = confirm_dialog
        return
    
    selected_dir = qw.get_selected_df().index[0]
    sel_dir = selected_dir.split('"')[1].split('notebooks/notebooks/')[1]
    if w == del_button:
        sel_dir = get_del_dir(sel_dir)
        confirm_label.value = 'Delete all files in %s ?' % sel_dir
        op = del_project
    elif w == send_button:
        confirm_label.value = 'Send all files in %s to HydroShare?' % get_del_dir(sel_dir)
        op = send_project
    elif w == update_button:
        confirm_label.value = 'Update all files in %s from HydroShare (overwriting any local changes)?' % get_del_dir(sel_dir)
        op = update_project
    else:
        confirm_label.value = 'Make a new resource from the files in %s ?' % get_del_dir(sel_dir)
        op = copy_project

    hbox.children = confirm_dialog

def do_copy(hs, rid, sel_dir):
    # copy files to resource 'rid' from directory 'sel_dir'
    os.chdir(sel_dir)
    try:
        for dirpath, dnames, fnames in os.walk('.'):
            if dirpath.endswith('.ipynb_checkpoints'):
                continue
            # print('DIR=', dirpath)
            for dn in dnames:
                if dn != '.ipynb_checkpoints':
                    # print('dn=', dn)
                    try:
                        hs.hs.createResourceFolder(rid, pathname=dn)
                    except:
                        pass
                    print('.', end='')
            for f in fnames:
                #print('f=', f)
                if dirpath == '.':
                    hs.hs.addResourceFile(rid, f)
                    print('.', end='')
                    continue
                # Ugh. Copy then rename because the API is broken
                f =  os.path.join(dirpath, f).lstrip('./')
                #print("Copying", f)
                hs.hs.addResourceFile(rid, f, '_tmp_')
                print('.', end='')
                options = {
                    "source_path": "_tmp_",
                    "target_path": f
                }
                hs.hs.resource(rid).functions.move_or_rename(options)
                print('.', end='')
        print("DONE")
        os.chdir(os.path.expanduser('~/notebooks'))
        return 1
    except:
        print('ERROR: Copy Failed')
        return 0

def send_project():
    # send files to hydroshare
    global hs
    outbox.clear_output()
    with outbox:
        rid = sel_dir.split('/')[-3]
        # delete current files
        resource = hs.hs.resource(rid)
        r = resource.files.all().json()
        if 'results' not in r:
            print(r)
            return
        for f in r['results']:
            url = f['url']
            f = url.split('contents/')[1]
            #print('deleting', f)
            print('.', end='')
            hs.hs.deleteResourceFile(rid, f)

        # upload new files
        if do_copy(hs, rid, sel_dir):
            # set modified date on contents
            os.system('touch %s' % sel_dir) 
            qw.df = build_df(hs)

def copy_project():
    global hs
    outbox.clear_output()
    with outbox:
        rid = sel_dir.split('/')[-3]
        meta = hs.getResourceMetadata(rid)
        abstract = meta.abstract \
                    + '\n\n[Modified in JupyterHub on %s]\n' \
                    % datetime.datetime.now()
        rid = hs.hs.createResource(resource_type=meta.resource_type,
                              title=meta.title,
                              abstract=abstract,
                              keywords=meta.keywords)
        print("Created new resource", rid) 
        if do_copy(hs, rid, sel_dir):
            print("Downloading new version")
            download_dir = os.environ.get('JUPYTER_DOWNLOADS', 'Downloads')
            hs.hs.getResource(rid, destination=download_dir, unzip=True)
            print("DONE")
            qw.df = build_df(hs)
    
def update_project():
    outbox.clear_output()
    rid = sel_dir.split('/')[-3]
    with outbox:
        print("UPDATING", sel_dir)
        download_dir = os.environ.get('JUPYTER_DOWNLOADS', 'Downloads')
        hs.hs.getResource(rid, destination=download_dir, unzip=True)
        # set modified date on contents
        os.system('touch %s' % sel_dir) 
        print('DONE')
    qw.df = build_df(hs)

def new_project():
    outbox.clear_output()
    rid = hs.hs.createResource(resource_type='GenericResource', title='New Resource')
    with outbox:
        print("Created new resource", rid) 
        with open('/tmp/README' , 'w') as fp:
            fp.write("Go to https://www.hydroshare.org/resource/%s/ and edit the metadata (title, abstract, etc).\n" % rid)
            fp.write("After editing metadata, click on the UPDATE button to update the local copy.\n")
        hs.hs.addResourceFile(rid, '/tmp/README', 'README')
        print('Downloading local copy...  ', end='')
        download_dir = os.environ.get('JUPYTER_DOWNLOADS', 'Downloads')
        hs.hs.getResource(rid, destination=download_dir, unzip=True)
        print("DONE")
    qw.df = build_df(hs)
    
def del_project():
    # delete local project
    global sel_dir
    shutil.rmtree(sel_dir)
    qw.df = build_df(hs)
    
def cancel_op(_):
    # cancel
    global sel_dir, op
    sel_dir = ''
    op = None
    hbox.children = buttons
    qw.change_selection([])
    
def ok_clicked(_):
    global op, sel_dir
    if op is not None:
        op()
    cancel_op('')
    
ok_but.on_click(ok_clicked)
cancel_but.on_click(cancel_op)

In [6]:
def get_last_time(path):
    files =  glob.glob('%s/**' % path, recursive=True)
    content_files = [f for f in files if os.path.isfile(f)]
    if content_files == []:
        return '', 0
    size = int(sum(os.path.getsize(x) for x in content_files)/1024)
    latest_file = max(content_files, key=os.path.getmtime)
    ltime = time.strftime("%Y-%m-%d", time.localtime(os.path.getmtime(latest_file)))
    return ltime, size

def get_status(hs, rid, contents):
    resource_date = None
    try:
        meta = hs.getResourceMetadata(rid)
        for d in meta.dates:
            resource_date = d['start_date']
            if d['type'] == 'modified':
                break
    except:
        return '<div data-toggle="popover" title="Deleted" style="color:red;">X</div>'
            
    if resource_date is None:
        return '<div data-toggle="popover" title="unknown">?</div>'
    resource_date = resource_date.rstrip('Z')
    dx = dateutil.parser.parse(resource_date)
    dy = datetime.datetime.fromtimestamp(os.path.getmtime(contents))
    if dx > dy:
        return '<div data-toggle="popover" title="Modified" style="color:orange;">M</div>'
    return '<div data-toggle="popover" title="Current">C</div>'

def get_title(f):
    g = rdflib.Graph().parse(f)
    title = 'unknown'
    for s,p,o in g:
        if p.title().endswith('/Title'):
            title = o.toPython()
            break
    g.close()
    return title

def build_df(hs):
    files = glob.glob('data/*/data/resourcemetadata.xml')
    files.extend(glob.glob('data/*/*/data/resourcemetadata.xml'))

    data = OrderedDict({'Name': [],
            'ID': [],
            'Status': [],
            'Last': [],
            'Size (KB)': []
           })
    
    for f in files:
        rid = f.split('/')[1]
        content_dir = os.path.join(os.path.dirname(f), 'contents')
        # generate absolute path to content for href
        cdir = '/hub/user-redirect/notebooks/notebooks/' + content_dir
        ltime, size = get_last_time(content_dir)
        data['Name'].append('<a href="{}" target="_blank">{}</a>'.format(cdir, get_title(f)))
        data['ID'].append(rid)
        data['Last'].append(ltime)
        data['Status'].append(get_status(hs, rid, content_dir))         
        data['Size (KB)'].append(size)

    df = pd.DataFrame.from_dict(data)
    return df.set_index('Name')

In [7]:
outbox = Output()
hbox = HBox(buttons)
with outbox:
    global hs
    hs = hydroshare.hydroshare()
    
df = build_df(hs)
col_defs = {
    'Name': {'width': 500},
    'ID': {'width': 250},
    'Last': {'width': 100},
    'Size (KB)': {'width': 80},
    'Status': {'width': 25}
}
qw = qgrid.show_grid(df, 
                     grid_options={'editable': False,'forceFitColumns': True}, 
                     column_definitions=col_defs)
qw.on('selection_changed', select_cb)
del_button.on_click(but_clicked)
send_button.on_click(but_clicked)
copy_button.on_click(but_clicked)
update_button.on_click(but_clicked)
new_button.on_click(but_clicked)

<div class="mytext" style="background-color:#BCBCBC;max-width: 100%">
<a href="." data-toggle="popover" title="Open Notebook Files" style="horizontal-align:middle;text-decoration:none;color:white"><i class="fa fa-file"></i>
Click Here to go to your Home Directory</a>
</div>

## Welcome to the HydroShare Notebook Server 

For more information about what you can do with Jupyter and HydroShare, see
* [Welcome](/user-redirect/notebooks/notebooks/Welcome.ipynb)

Hydroshare Resources you have downloaded are listed below. 

In [8]:
display(qw)
display(hbox)
display(outbox)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Output()